In [22]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.autograd import Variable
from scipy.stats import truncnorm
import torch.nn.init as init

In [23]:
batch_size = 50
epochs = 40000
log_interval =10

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [25]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size= batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size= batch_size, shuffle=True)

In [26]:
len(train_loader)

1200

In [27]:
def kaiming_init(m):
    if isinstance(m, (nn.Linear)):
        init.kaiming_normal(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.1)

In [28]:
n_hidden_1 = 500 
n_hidden_2 = 501 
n_hidden_3 = 503 

class VAE(nn.Module):
    def __init__(self, nz):
        super(VAE, self).__init__()

        self.have_cuda = False
        self.nz = nz

        self.fc1 = nn.Linear(784, n_hidden_1)
        self.fc2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.fc3 = nn.Linear(n_hidden_2, n_hidden_3)
        
        self.fc31 = nn.Linear(n_hidden_3, nz)
        self.fc32 = nn.Linear(n_hidden_3, nz)

        self.fc4 = nn.Linear(nz,  n_hidden_1)
        self.fc5 = nn.Linear(n_hidden_1, n_hidden_2)
        self.fc6 = nn.Linear(n_hidden_2, n_hidden_3)
        self.fc61 = nn.Linear(n_hidden_3, 784)

        self.softplus = nn.Softplus()
        self.sigmoid = nn.Sigmoid()
        
        self.weight_init()
        
    def weight_init(self):
        for block in self._modules:
                kaiming_init(block)

    def encode(self, x):
        h1 = self.softplus(self.fc1(x))
        h2 = self.softplus(self.fc2(h1))
        h3 = self.softplus(self.fc3(h2))
        return self.fc31(h3), self.fc32(h3)
    

    def decode(self, z):
        h1 = self.softplus(self.fc4(z))
        h2 = self.softplus(self.fc5(h1))
        h3 = self.softplus(self.fc6(h2))
        deconv_input = self.sigmoid(self.fc61(h3))
        # print("deconv_input", deconv_input.size())
        # deconv_input = deconv_input.view(-1,1024,1,1)
        # print("deconv_input", deconv_input.size())
        return deconv_input

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = torch.cuda.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def forward(self, x):
        x = x.view(-1,784)
        mu, logvar = self.encode(x)
        # print("mu, logvar", mu.size(), logvar.size())
        z = self.reparametrize(mu, logvar)
        # print("z", z.size())
        decoded = self.decode(z)
        # print("decoded", decoded.size())
        return decoded, mu, logvar


In [29]:
model = VAE(nz=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [30]:
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [31]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader)))

In [32]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(devzice)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [33]:
for batch_idx, (data, _) in enumerate(train_loader):
    data = data.to(device)
    optimizer.zero_grad()
    for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 2).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'results/sample_' + str(epoch) + '.png')
    recon_batch, mu, logvar = model(data)

Train Epoch: 1 [0/60000 (0%)]	Loss: 555.683359
Train Epoch: 1 [500/60000 (1%)]	Loss: 228.405000
Train Epoch: 1 [1000/60000 (2%)]	Loss: 217.930098
Train Epoch: 1 [1500/60000 (2%)]	Loss: 206.003594
Train Epoch: 1 [2000/60000 (3%)]	Loss: 203.452734
Train Epoch: 1 [2500/60000 (4%)]	Loss: 207.618320
Train Epoch: 1 [3000/60000 (5%)]	Loss: 203.466602
Train Epoch: 1 [3500/60000 (6%)]	Loss: 211.164824
Train Epoch: 1 [4000/60000 (7%)]	Loss: 206.908887
Train Epoch: 1 [4500/60000 (8%)]	Loss: 202.884570
Train Epoch: 1 [5000/60000 (8%)]	Loss: 193.938340
Train Epoch: 1 [5500/60000 (9%)]	Loss: 192.467324
Train Epoch: 1 [6000/60000 (10%)]	Loss: 193.682266
Train Epoch: 1 [6500/60000 (11%)]	Loss: 200.152500
Train Epoch: 1 [7000/60000 (12%)]	Loss: 189.079238
Train Epoch: 1 [7500/60000 (12%)]	Loss: 184.916426
Train Epoch: 1 [8000/60000 (13%)]	Loss: 175.483887
Train Epoch: 1 [8500/60000 (14%)]	Loss: 201.925742
Train Epoch: 1 [9000/60000 (15%)]	Loss: 180.089902
Train Epoch: 1 [9500/60000 (16%)]	Loss: 188.989

NameError: name 'devzice' is not defined